In [1]:
"""
Evaluation
"""

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
import torchvision.datasets as dset
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD
import os


/home/seonhye/.local/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [3]:
# Specify the cuda setting if you needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 

Home_directory = ''
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [5]:

dataset = dset.ImageFolder(root=DeepTaster_forder +'images/test',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [6]:
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
c = torch.Tensor(state_dict['center']).to(device)
net=torch.load(DeepTaster_forder +'models/classifier/deepsvdd.pt').to(device)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','r')
threshold=float(threshold_file.readline())
threshold_file.close()

In [8]:
# NOTE THAT THE ORDER OF RESULTS IS THREE CIFAR10 MODELS > THREE MNIST MODELS > THREE TINY-IMAGENET MODELS
# GROUND TRUTH RESULTS: STOLEN, STOLEN, STOLEN, BENIGN, BENIGN, BENIGN, BENIGN, BENIGN, BENIGN
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
detection_rate =[]
architecture_list = ["DenseNet", "ResNet","VGG"]
dataset = ["CIFAR10", "MNIST","Tiny-ImageNet","ImageNet"]
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            if y[0]==0:
                detection_rate.append(100 if attack_probability >=144 else 0)
            else:
                detection_rate.append(100 if attack_probability < 144 else 0)
            temp=0
            attack_probability=0

print("MAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Architecture","Attack Prob.","Copy Detection (%)"))
for i in range(3):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Stolen", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))
for i in range(3,12):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Benign", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))


MAA results

Victim          Suspect         Ground Truth    Architecture    Attack Prob.    Copy Detection (%)
CIFAR10         CIFAR10         Stolen          DenseNet        0.99            100            
CIFAR10         CIFAR10         Stolen          ResNet          1.0             100            
CIFAR10         CIFAR10         Stolen          VGG             0.99            100            
CIFAR10         MNIST           Benign          DenseNet        0.0             100            
CIFAR10         MNIST           Benign          ResNet          0.03            100            
CIFAR10         MNIST           Benign          VGG             0.0             100            
CIFAR10         Tiny-ImageNet   Benign          DenseNet        0.01            100            
CIFAR10         Tiny-ImageNet   Benign          ResNet          0.01            100            
CIFAR10         Tiny-ImageNet   Benign          VGG             0.28            100            
CIFAR10         ImageNet